# Analisis sobre publicaciones de alquileres en CABA

El objetivo de este notebook es hacer analisis sobre los alquileres de vivienda de las paginas de zonaprop y argenprop. Los datos fueron scrapeados por mi entonces pueden no reflarar la realidad del sitio, ademas es muy comun que los datos sean erroneos por ejemplo que el inmueble en cuestion es en dolares pero en la publicacion esta en dolares.
Tampoco voy a considerar la "calidad" del inmueble, osea si tiene balcon, si da a la calle o al pulmon, amenities del edificio, la antiguedad del edificio, etc.

In [122]:
import numpy as np
import sqlite3
import pandas as pd
import plotly.express as px

In [123]:
conn = sqlite3.connect("./scan_house.db")

In [124]:
df = pd.read_sql("SELECT * FROM props", conn)

In [125]:
df

,id,title,url,m2_total,m2_cub,direction,price,expens,currency,location,rooms,bedrooms,date_created,date_updated
0,1,Departamento en Alquiler en Palermo Chico con ...,https://www.zonaprop.com.ar/propiedades/depart...,285.0,1.0,Libertador al 2600,3500.0,140.000,USD,palermo,6.0,4.0,2023-04-02 17:44:01,None
1,2,Casa - Palermo Chico,https://www.zonaprop.com.ar/propiedades/casa-p...,360.0,360.0,Ombu al 3000,0.0,0.000,Consultar,palermo,6.0,3.0,2023-04-02 17:44:03,None
2,3,Palacio Bellini! Duplex Increíble con Terraza,https://www.zonaprop.com.ar/propiedades/palaci...,122.0,104.0,Republica Arabe Siria 3000,3000.0,1.000,USD,palermo,2.0,1.0,2023-04-02 17:44:04,None
3,4,"3 Amb. Torre Weik a Estrenar, Inmejorable Ubic...",https://www.zonaprop.com.ar/propiedades/3-amb....,96.0,75.0,Arcos 1440 - Caba.,1700.0,29.800,USD,belgrano,3.0,2.0,2023-04-02 17:44:09,None
4,5,Montevideo y Quintana,https://www.zonaprop.com.ar/propiedades/montev...,109.0,102.0,Montevideo al 1700,1200.0,69.421,USD,recoleta,4.0,3.0,2023-04-02 17:44:13,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7389,7403,PH Monoambiente - Amoblado - Paternal,https://www.argenprop.com/ph-en-alquiler-en-pa...,25.0,25.0,"Oroño, nicasio 2100, piso pb",200.0,1.000,USD,paternal,1.0,1.0,2023-04-17 02:25:46,None
7390,7404,Alquiler 2 ambientes Puerto Madero Torre El M...,https://www.argenprop.com/departamento-en-alqu...,66.0,66.0,Juana manso 1100,1100.0,1.000,USD,centro,2.0,1.0,2023-04-17 02:25:52,None
7391,7405,Departamento - Caballito,https://www.argenprop.com/departamento-en-alqu...,27.0,27.0,Campichuelo al 100,80000.0,22000.000,$,caballito,2.0,1.0,2023-04-17 02:25:59,None
7392,7406,"PH al frente, 2 ambientes , patio cubierto 3x3...",https://www.argenprop.com/ph-en-alquiler-en-ve...,45.0,45.0,Moliere 1000,140000.0,1.000,$,versalles,2.0,1.0,2023-04-17 02:26:20,None


Primero arranco limpiando un poco la informacion. Me voy a quedar con los datos que tengan superficie mayor y el precio entre 10 y 1 millon, porque departamentos menores a 20 son muy escasos y si tienen precio menor a 10 no esta correctamente cargado el precio o si el precio es mayor a 1 millon entonces  lo mas probable es que sea un anuncio de venta, que no es lo que quiero analizar ahora.

In [140]:
mask = (df["price"] > 10) & \
        (df["price"] < 1_000_000) & \
       (df["m2_cub"] > 20) 
df = df[mask]

Para empezar veamos como estan distribuidos los datos segun el barrio

In [141]:
px.histogram(
    df,
    x="location",
    barmode="group",
    title="Resultados por barrio",
    ).update_xaxes(categoryorder="total descending")

Aca se ve que los alquileres no estan distribuidos de forma uniforme en los barrios con . Entonces me voy a quedar con los barrios con mas de 40 publicaciones para seguir con el analisis

In [142]:
top_locations = df["location"].value_counts()[df["location"].value_counts() > 40]
df2 = df[df["location"].isin(top_locations.keys())]

In [143]:
px.histogram(
    df2,
    x="location",
    barmode="group",
    title="Resultados por barrio",
    ).update_xaxes(categoryorder="total descending")

Las publicaciones hoy en dia pueden estar en pesos o en dolares, veamos como se distribuyen esas operaciones

In [144]:
px.histogram(
    df2,
    x="currency",
    title="Resultados por moneda",
    histnorm='percent'
    ).update_xaxes(categoryorder="total descending")

Entonces vemos que la mayoria de las publicaciones (57%) son en dolares

In [145]:
px.histogram(
    df2,
    x="location",
    barmode="group",
    title="Resultados por barrio",
    color="currency"
    ).update_xaxes(categoryorder="total descending")

Aca vemos como predominan las publicaciones en dolar para los barrios de palermo, belgrno recoleta y puerto madero. Pero en barrios como barrio norte, caballito, nuñez estan igualados o predomina las publicaciones en pesos.

Antes de seguir con el analisis voy a sepearar el analisis de alquileres en pesos y dolares.

In [177]:
df2["p/m"] = df2["price"] / df2["m2_cub"] 
df2_pesos = df2[df2["currency"]=="$"]
top_locations_pesos = df2_pesos["location"].value_counts()[
    df2_pesos["location"].value_counts() > 40
]
df2_pesos = df2_pesos[df2_pesos["location"].isin(top_locations_pesos.keys())]

/tmp/ipykernel_63949/3911704501.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [209]:
median_price_pesos = df2_pesos.groupby("location")["p/m"].median().reset_index()
px.bar(
    median_price_pesos,
    x="location",
    y="p/m",
    labels={"x": "bario", "value": "p/m"}
).update_xaxes(categoryorder="total descending")

In [179]:
px.scatter(
    df2_pesos,
    x="m2_cub",
    y="price",
    color="location"
    )

Ahora con los datos mas limpios si se observa el grafico barrio por barrio se ve que hay una pendiente para cada uno. Sigue habiendo avisos alejados de la mediana que deben ser en dolares por su bajo valor.

En el analisis de valor de precio por m2 se ven cosas esperables, barrios como palermo , nuñes, belgrano entre los mas caros y otros como liniers, villa luro, floresta como los mas baratos.

Para el bario de centro hay que tener en cuenta que hay muchos avisos que son alquiler de oficinas o solamente para uso comercial, que no es el objetivo del analisis que quiero hacer.

In [185]:
df2_dolar = df2[df2["currency"]=="USD"]
top_locations_dolar = df2_dolar["location"].value_counts()[
    df2_dolar["location"].value_counts() > 40
]
df2_dolar = df2_dolar[df2_dolar["location"].isin(top_locations_dolar.keys())]

In [194]:
median_price_dolar = df2_dolar.groupby("location")["p/m"]\
.median().reset_index()
px.bar(
    median_price_dolar,
    x="location",
    y="p/m",
    labels={"x": "bario", "value": "p/m"}
).update_xaxes(categoryorder="total descending")

Bastante curioso el caso de centro que antes era uno de los barrios mas baratos ahora se encuentra entre los mas caros. 

Suponiendo que los dos tipos de inmuebles tanto en pesos como en dolares son de igual calidad veamos cuanto daria el dolar

In [202]:
top_locations = list(set(top_locations_dolar.keys()) & set(top_locations_pesos.keys()))

In [240]:
a =median_price_pesos[median_price_pesos["location"].isin(top_locations)].reset_index()
b= median_price_dolar[median_price_dolar["location"].isin(top_locations)].reset_index()
a["dolar"] = a["p/m"]/b["p/m"]

In [244]:
a.sort_values(by="dolar")

,index,location,p/m,dolar
3,7,centro,2023.809524,105.019305
7,16,san telmo,2379.348193,157.826111
5,13,palermo,3229.166667,182.986111
2,6,caballito,2638.888889,201.050347
6,14,recoleta,2766.194332,203.772144
1,4,belgrano,3000.000000,204.545455
4,12,nuñez,3139.534884,208.892841
0,3,barrio norte,2727.272727,210.389610
8,21,villa urquiza,2954.545455,260.000000


Curiosamente el "dolar" va desde 105 a 260, cuando el dolar oficial esta a 200 y el blue alrededor (al dia de 17/04)

In [247]:
px.scatter(
    df2_dolar,
    x="m2_cub",
    y="price",
    color="location",
    hover_data=["currency"]
    )

Aca veo que hay muchos resultados por arriba de 50k, que deben ser ventas. Entonces voy a filtrar esos casos

In [253]:
mask= df2_dolar["price"] < 2000
df2_dolar[mask]
px.scatter(
    df2_dolar[mask],
    x="m2_cub",
    y="price",
    color="location",
    hover_data=["currency"]
    )

Vamos a volver a filtrar los datos los datos filtrados por el precio

In [254]:
median_price_dolar = df2_dolar[mask].groupby("location")["p/m"]\
.median().reset_index()
px.bar(
    median_price_dolar,
    x="location",
    y="p/m",
    labels={"x": "bario", "value": "p/m"}
).update_xaxes(categoryorder="total descending")

In [258]:
a =median_price_pesos[median_price_pesos["location"].isin(top_locations)].reset_index()
b= median_price_dolar[median_price_dolar["location"].isin(top_locations)].reset_index()
a["dolar"] = a["p/m"]/b["p/m"]
a.sort_values(by="dolar")

,index,location,p/m,dolar
3,7,centro,2023.809524,126.629423
7,16,san telmo,2379.348193,166.554374
5,13,palermo,3229.166667,193.750000
2,6,caballito,2638.888889,203.092898
6,14,recoleta,2766.194332,211.674001
4,12,nuñez,3139.534884,222.503136
1,4,belgrano,3000.000000,232.500000
0,3,barrio norte,2727.272727,233.766234
8,21,villa urquiza,2954.545455,261.688312
